## Setup

In [ ]:
from specific import *

### Retrieve previous results from the 'model' notebook

In [ ]:
X_train, X_test, y_train, y_test = data_split_cache.load()
results, rf = cross_val_cache.load()

#### Get Dask Client

In [ ]:
client = get_client()
# client = Client(n_workers=1, threads_per_worker=8, resources={'threads': 8})
client

In [ ]:
rf_fut = client.scatter(rf, broadcast=True)
X_fut = client.scatter(X_train, broadcast=True)

## ALE Plotting

### Worldwide

In [ ]:
world_ale_1d_cache = SimpleCache("world_ale_1d", cache_dir=CACHE_DIR)

# world_ale_1d_cache.clear()


@world_ale_1d_cache
def get_world_ale_1d():
    n_threads = 16
    ale_fs = [
        client.submit(
            add_common_path_deco(save_ale_plot_1d_with_ptp),
            model=rf_fut,
            X_train=X_fut,
            column=column,
            n_jobs=n_threads,
            monte_carlo_rep=1000,
            resources={"threads": n_threads},
        )
        for column in X_train.columns
    ]

    for ale_f in tqdm(
        dask.distributed.as_completed(ale_fs),
        total=len(ale_fs),
        unit="plot",
        desc="Calculating 1D ALE plots",
        smoothing=0,
    ):
        if ale_f.status == "error":
            print(ale_f.result())

    ptp_values = {}
    mc_ptp_values = {}

    for column, ale_f in zip(X_train.columns, ale_fs):
        ptp_values[column], mc_ptp_values[column] = ale_f.result()
    return ptp_values, mc_ptp_values


ptp_values, mc_ptp_values = get_world_ale_1d()

### Run Non-MC runs manually (just for the plots)

In [ ]:
n_threads = 16
ale_fs = [
    client.submit(
        add_common_path_deco(save_ale_plot_1d_with_ptp),
        model=rf_fut,
        X_train=X_fut,
        column=column,
        n_jobs=n_threads,
        monte_carlo=False,
        resources={"threads": n_threads},
    )
    for column in X_train.columns
]

for ale_f in tqdm(
    dask.distributed.as_completed(ale_fs),
    total=len(ale_fs),
    unit="plot",
    desc="Calculating 1D Non-MC ALE plots",
    smoothing=0,
):
    if ale_f.status == "error":
        print(ale_f.result())

## PDP Plotting

### Worldwide

In [ ]:
%time save_pdp_plot_1d(rf, X_train, 'Dry Day Period -12 - 0 Month', n_jobs=32)

### Sequentially Locally

In [ ]:
for column in tqdm(
    X_train.columns, unit="plot", desc="Calculating 1D PDP plots", smoothing=0.05
):
    figs, pdp_isolate_out, data_file = save_pdp_plot_1d(rf, X_train, column, n_jobs=32)
    for fig in figs:
        plt.close(fig)

### Using a Dask distributed Cluster

In [ ]:
%%time
n_threads = 32
pdp_fs = [
    client.submit(
        add_common_path_deco(save_pdp_plot_1d),
        model=rf_fut,
        X_train=X_fut,
        column=column,
        n_jobs=n_threads,
        resources={"threads": n_threads},
    )
    for column in X_train.columns
]

for pdp_f in tqdm(
    dask.distributed.as_completed(pdp_fs),
    total=len(pdp_fs),
    unit="plot",
    desc="Calculating 1D PDP plots",
    smoothing=0.04,
):
    if pdp_f.status == "error":
        print(pdp_f.result())